## Dependencies

In [1]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 9.4 MB/s eta 0:00:00


In [2]:
import os
import json
import timeit
import random

from tqdm.auto import tqdm

import pandas as pd
import torch

from pinecone import Pinecone, ServerlessSpec

## Load and Prepare Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/pinecone/pinecone_api_key.txt', 'r') as f:
  API_KEY = f.read().strip()

In [5]:
with open('/content/drive/MyDrive/ir_project_dataset/training13b.json', 'rb') as f:
  data_dict = json.load(f)

In [6]:
data_dict['questions'][0]['snippets'][0]

{'offsetInBeginSection': 131,
 'offsetInEndSection': 358,
 'text': 'Hirschsprung disease (HSCR) is a multifactorial, non-mendelian disorder in which rare high-penetrance coding sequence mutations in the receptor tyrosine kinase RET contribute to risk in combination with mutations at other genes',
 'beginSection': 'abstract',
 'document': 'http://www.ncbi.nlm.nih.gov/pubmed/15829955',
 'endSection': 'abstract'}

In [7]:
contexts = []

for q in data_dict['questions']:
  for context in q['snippets']:
    contexts.append(dict(context_text=context['text']))

In [8]:
contexts[0]

{'context_text': 'Hirschsprung disease (HSCR) is a multifactorial, non-mendelian disorder in which rare high-penetrance coding sequence mutations in the receptor tyrosine kinase RET contribute to risk in combination with mutations at other genes'}

In [9]:
contexts_df = pd.DataFrame(contexts)
contexts_df.head()

,context_text
0,Hirschsprung disease (HSCR) is a multifactoria...
1,"In this study, we review the identification of..."
2,"Coding sequence mutations in e.g. RET, GDNF, E..."
3,For almost all of the identified HSCR genes in...
4,Hirschsprung disease (HSCR) is a multifactori...


In [10]:
contexts_df.shape

(68860, 1)

## Initialize Pinecone

In [11]:
pc = Pinecone(api_key=API_KEY)
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [12]:
index_name = "bioasq-question-answering"

In [13]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
  # if does not exist, create index
  pc.create_index(
    index_name,
    dimension=768,
    metric='cosine',
    spec=spec
  )
  # wait for index to be initialized
  while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Retriever Model

Sentence Embeddings: https://huggingface.co/flax-sentence-embeddings/all_datasets_v3_mpnet-base

In [14]:
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Create Embeddings and Upsert/Insert to Pinecone

In [15]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(contexts_df), batch_size)):
  # find end of batch
  i_end = min(i+batch_size, len(contexts_df))
  # extract batch
  batch = contexts_df.iloc[i:i_end]
  # generate embeddings for batch
  emb = retriever.encode(batch["context_text"].tolist()).tolist()
  # get metadata
  meta = batch.to_dict(orient="records")
  # create unique IDs
  ids = [f"{idx}" for idx in range(i, i_end)]
  # add all to upsert list
  to_upsert = list(zip(ids, emb, meta))
  # upsert/insert these records to pinecone
  _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/1076 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 68928}},
 'total_vector_count': 68928}

In [16]:
def query_pinecone(query, top_k):
  # generate embeddings for the query
  xq = retriever.encode([query]).tolist()
  # search pinecone index for context passage with the answer
  xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
  return xc

In [59]:
query = "Is RANKL secreted from the cells?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': '50',
              'metadata': {'context_text': 'Activated human T cells express '
                                           'alternative mRNA transcripts '
                                           'encoding a secreted form of '
                                           'RANKL.'},
              'score': 0.553318918,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

## Generator Model

bart-lfqa model: https://huggingface.co/vblagoje/bart_lfqa

In [110]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

In [133]:
# from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

# def model_pipeline(query_and_docs):
#   model_name = "vblagoje/bart_lfqa"
#   tokenizer = AutoTokenizer.from_pretrained(model_name)
#   model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

#   model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

#   generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
#                                            attention_mask=model_input["attention_mask"].to(device),
#                                            min_length=64,
#                                            max_length=256,
#                                            do_sample=False,
#                                            early_stopping=True,
#                                            num_beams=8,
#                                            temperature=1.0,
#                                            top_k=None,
#                                            top_p=None,
#                                            eos_token_id=tokenizer.eos_token_id,
#                                            no_repeat_ngram_size=3,
#                                            num_return_sequences=1)

#   answer = tokenizer.batch_decode(generated_answers_encoded,
#                                   skip_special_tokens=True,
#                                   clean_up_tokenization_spaces=True)
#   return answer


In [111]:
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# MODEL_NAME = "deepset/roberta-base-squad2"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)

In [112]:
def format_query(query, context):
  # extract context_text from Pinecone search result and add the  tag
  # inputs to the model should be formated -> question: question context: <P> context <P> context...
  context = [f"<P> {m['metadata']['context_text']}" for m in context]
  # concatinate all context passages
  context = " ".join(context)
  # contcatinate the query and context passages
  query = f"question: {query} context: {context}"
  return query

In [113]:
query = format_query(query, result["matches"])
print(query)

question: Are long non coding RNAs spliced? context: <P> Activated human T cells express alternative mRNA transcripts encoding a secreted form of RANKL.


In [115]:
# def generate_answer(query, context):
#   results = []
#   for doc in context["matches"]:
#     results.append(doc["metadata"]["context_text"])
#   qa_model = pipeline(task="question-answering", model=generator, tokenizer=tokenizer, device=device)
#   return qa_model(question=query, context=' '.join(results))

In [118]:
def generate_answer(query):
  # tokenize the query to get input_ids
  inputs = tokenizer([query], max_length=1024, truncation=True, return_tensors="pt").to(device)
  # use generator to predict output ids
  ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
  # use tokenizer to decode the output ids
  answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  return answer

In [136]:
query = "Is the protein Papilin secreted?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
print(generate_answer(query))
# print(model_pipeline(query))

Papilin is a glycoprotein, meaning it is an extracellular matrix glycoprotein. It is secreted into the medium by hemocytes.


In [120]:
for doc in context["matches"]:
  print(doc["metadata"]["context_text"], end='\n---\n')

Papilins are extracellular matrix proteins 
---
Papilin is an extracellular matrix glycoprotein 
---
Ghrelin is a 28-amino acid peptide secreted mainly by the stomach.
---
 Collagen IV, laminin, glutactin, papilin, and other extracellular matrix proteins were made primarily by hemocytes and were secreted into the medium. 
---
Ghrelin is a gut peptide composed of 28 amino acids mostly secreted in the gastric fundus mucosa.
---


In [137]:
query = "Are long non coding RNAs spliced?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
print(generate_answer(query))
# print(model_pipeline(query))

Yes, long non coding RNAs (LNCRNAs) can be spliced, but they are not normally spliced. The reason for this is that LNCRNAs are not


In [122]:
for doc in context["matches"]:
  print(doc["metadata"]["context_text"], end='\n---\n')

For alternative exons and long noncoding RNAs, splicing tends to occur later, and the latter might remain unspliced in some cases.
---
 Long noncoding RNAs (lncRNAs) are more than 200 nucleotides in length and lack transcriptional ability.
---
Natural SINEUPs suggest that embedded Transposable Elements may represent functional domains in long non-coding RNAs
---


## Evaluation: ROUGE Metric

In [31]:
questions_and_answers = []

for q in data_dict['questions']:
  questions_and_answers.append(dict(question=q['body'],
                                    answer=q['ideal_answer'][0]))

qna_df = pd.DataFrame(questions_and_answers)
qna_df.head()

,question,answer
0,Is Hirschsprung disease a mendelian or a multi...,"Coding sequence mutations in RET, GDNF, EDNRB,..."
1,List signaling molecules (ligands) that intera...,The 7 known EGFR ligands are: epidermal growt...
2,Is the protein Papilin secreted?,"Yes, papilin is a secreted protein"
3,Are long non coding RNAs spliced?,Long non coding RNAs appear to be spliced thro...
4,Is RANKL secreted from the cells?,Receptor activator of nuclear factor κB ligand...


In [100]:
valid_df = qna_df.sample(frac=0.1, random_state=42)
valid_df.shape

(539, 2)

In [123]:
valid_questions = valid_df['question'].tolist()

valid_questions[:5]

['What is small-activating RNA?',
 'Can the CEP290 gene mutations be targeted by AAV-mediated gene therapy?',
 'Is there an approved vaccine against Helicobacter pylori?',
 'Which company produces Glybera?',
 'Cerliponase alfa is apprived for treatment of which disease?']

In [124]:
valid_df['answer'].tolist()[:5]

['small activating RNAs are double stranded RNAs (dsRNAs) that target gene promoters and trigger gene activation.',
 'The large size of the CEP290 gene prevents its use in adeno-associated virus (AAV)-mediated gene augmentation therapy.',
 'No, there is no approved vaccine against Helicobacter pylori.',
 'Glybera is a product of Chiesi Pharma.',
 'Cerliponase alfa is a recombinant human tripeptidyl peptidase-1 (TPP1) approved for use in patients with neuronal ceroid lipofuscinosis type 2 (CLN2), a paediatric neurodegenerative disease caused by a deficiency in TPP1.']

In [126]:
predicted_answers = []
for question in valid_questions:
  context = query_pinecone(question, top_k=3)
  query = format_query(question, context["matches"])
  answer = generate_answer(query)
  predicted_answers.append(answer)

In [127]:
predicted_answers[:5]

["Small-activating RNA is a class of RNA that is non-coding, meaning it doesn't have a specific function. It can bind to mRNA and inhibit its degradation or inhibiting",
 'The CEP290 gene is a very large gene, so it would be very difficult to target it with a gene therapy.',
 'There is an approved vaccine against Helicobacter pylori, but it is not effective. There is also a vaccine against Hepatitis B, but it is not effective. There are',
 'Glybera is produced by Bayer. Bayer is a subsidiary of Bayer. Bayer is a subsidiary of Bayer. Bayer is a subsidiary of Bayer. Bayer is a subsidiary of Bayer. Bayer',
 'Cerliponase Alfa is a recombinant human tripeptidyl peptidase-1 (TPP1) being developed by BioMarin Pharmaceutical Inc. for use in']

In [128]:
valid_df['predicted_answer'] = predicted_answers
valid_df.head()

,question,answer,predicted_answer
2810,What is small-activating RNA?,small activating RNAs are double stranded RNAs...,Small-activating RNA is a class of RNA that is...
2316,Can the CEP290 gene mutations be targeted by A...,The large size of the CEP290 gene prevents its...,"The CEP290 gene is a very large gene, so it wo..."
5085,Is there an approved vaccine against Helicobac...,"No, there is no approved vaccine against Helic...",There is an approved vaccine against Helicobac...
3084,Which company produces Glybera?,Glybera is a product of Chiesi Pharma.,Glybera is produced by Bayer. Bayer is a subsi...
3039,Cerliponase alfa is apprived for treatment of ...,Cerliponase alfa is a recombinant human tripep...,Cerliponase Alfa is a recombinant human tripep...


In [129]:
ideal_answers = valid_df['answer'].tolist()
predicted_answers = valid_df['predicted_answer'].tolist()

In [44]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [45]:
import evaluate

In [46]:
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

In [130]:
rouge_scores = []
bleu_scores = []

for pred_answer, ideal_answer in zip(predicted_answers, ideal_answers):
  rouge_scores.append(rouge.compute(predictions=[pred_answer], references=[ideal_answer]))
  # bleu_scores.append(bleu.compute(predictions=[pred_answer], references=[ideal_answer]))

In [131]:
rouge_scores[0]

{'rouge1': 0.1702127659574468,
 'rouge2': 0.04444444444444444,
 'rougeL': 0.1702127659574468,
 'rougeLsum': 0.1702127659574468}

In [132]:
rouge1_scores = [score['rouge1'] for score in rouge_scores]
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge1_score

0.28547043977262304